# Выбор локации для скважины

Допустим, вы работаете в добывающей компании. Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [ ]:
#загрузим необходимые библиотеки
import pandas as pd
import plotly.express as px
import numpy as np
from scipy import stats as st

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [ ]:
#загружаем необходимые датасеты локально или с ЯП
try:
    geo0 = pd.read_csv('geo_data_0.csv')
    geo1 = pd.read_csv('geo_data_1.csv')
    geo2 = pd.read_csv('geo_data_2.csv')
except:
    geo0 = pd.read_csv('/datasets/geo_data_0.csv')
    geo1 = pd.read_csv('/datasets/geo_data_1.csv')
    geo2 = pd.read_csv('/datasets/geo_data_2.csv')

In [ ]:
#выведем головы таблиц
display(geo0.head(), geo1.head(), geo2.head())

In [ ]:
#взглянем на данные таблиц
display(geo0.info(), geo1.info(), geo2.info())

In [ ]:
#и посмотрим на статистику по таблицам
display(geo0.describe(),geo1.describe(),geo2.describe())

Данные выглядят вполне готовыми к моделированию. Пропусков нет. Типы данных подходящие. Столбец с ID скважин нам не пригодится.

In [ ]:
#посмотрим на корреляцию фичей и таргета
display(geo0.drop('id',axis=1).corr(), geo1.drop('id',axis=1).corr(), geo2.drop('id',axis=1).corr())

Фичи друг с другом никак не коррелируют. Зато во втором регионе фича f2 напрямую влияет на количество продукта в скважине.

In [ ]:
#выведем матрицу корреляции для второго региона
fig = px.scatter_matrix(geo1.drop('id',axis=1))
fig.show()

Выглядит так, что с данными уже кто-то (или что-то) поработал. f2 максимально приближено к целым значениям: 0, 1, 2, 3, 4, 5, 6 и за каждую 1 ~ 26 едениц продукта. Было бы неплохо уточнить как такое могло получится.

In [ ]:
#создадим колонки с номерами регионов в каждом датасете
geo0['geo'] = 0
geo1['geo'] = 1
geo2['geo'] = 2

In [ ]:
#и сложим все наши датасеты в один для возможности применения функции
data = pd.concat([geo0, geo1, geo2], ignore_index=True)
display(data.info(), data.sample(20))

## Обучение и проверка модели

In [ ]:
#запишем в константу наше семя
SEED = np.random.RandomState(12345)

#создадим таблицу для записи наших предсказаний модели и валидных целей
predictions = pd.DataFrame()

#создадим функцию, принимающую таблицу и настройки модели
def all_pred(df, use_scale=False, use_normalize=False):
    #цикл по всем трём регионам
    for i in range(0,3):
        data_geo = df.loc[df['geo'] == i]
        X = data_geo.drop(['id','product','geo'], axis=1)
        y = data_geo['product']
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=SEED)
        if use_scale == True:
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_valid = scaler.transform(X_valid)
        else:
            pass
        model = LinearRegression(n_jobs=-1, normalize=use_normalize)
        model.fit(X_train, y_train)
        prediction = model.predict(X_valid)
        predictions['geo'+str(i)+'_pred'] = prediction
        predictions['geo'+str(i)+'_valid'] = y_valid.reset_index(drop=True)
        MSE = mean_squared_error(y_valid, prediction)
        RMSE = MSE ** 0.5
        print('\nСредний запас предсказанного сырья по региону {} = составил {:.2f} тыс. баррелей'.format(i,prediction.mean())) 
        print('RMSE составил {:.2f} тыс. баррелей'.format(RMSE))
        
        fig = px.histogram(predictions, x=['geo'+str(i)+'_pred', 'geo'+str(i)+'_valid'], marginal='box')
        fig.show()
    return predictions

In [ ]:
#применим функцию к нашей общей таблице без масштабирования и нормализации
all_pred(data)

Как мы видим модель отлично отработала во втором регионе, где RMSE меньше 1 тыс баррелей. А вот в первом и третьем регионе RMSE составил почти 40 тыс при средних 90 тыс.

In [ ]:
#попробуем запустить модель с масштабированием и нормализацией
all_pred(data, True, True)

Лучше не стало. Второй регион выглядит куда более предсказуемым для нашей модели и соотвественно риски там должны быть ниже.

## Подготовка к расчёту прибыли

In [ ]:
#посчитаем сколько необходимо выработать с одной скважины в среднем при условии бюджета в 10 млрд рублей и возвожности разработать только 200 из всех
1.0e+7 / 200 / 450

In [ ]:
#сохраним данные нашей новой таблицы в отдельные переменные
geo0_pred = predictions['geo0_pred']
geo0_valid = predictions['geo0_valid']
geo1_pred = predictions['geo1_pred']
geo1_valid = predictions['geo1_valid']
geo2_pred = predictions['geo2_pred']
geo2_valid = predictions['geo2_valid']

In [ ]:
#выведем среднее по каждому региону
display(geo0_valid.mean(), geo1_valid.mean(), geo2_valid.mean())

Ни в одном регионе средние значения не дотягивают до 111 тыс баррелей.

In [ ]:
#посмотрим чисто ради интереса, какие средние значения были бы, если бы мы точно знали какие 200 скважин принесут нам больше всего продукта
display(geo0_valid.sort_values(ascending=False).head(200).mean(), geo1_valid.sort_values(ascending=False).head(200).mean(), geo2_valid.sort_values(ascending=False).head(200).mean())

Почти ровно вдвое больше и мы были бы в плюсе в любом регионе.

In [ ]:
#запишем константы для расчёта прибыли в тысячах рублей
PRICE_PER_PRODUCT = 450
BUDGET = 10000000

In [ ]:
#запишем функцию для расчёта прибыли
def profit(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    revenue =  PRICE_PER_PRODUCT * selected.sum()
    return revenue - BUDGET

Проверим выводы для каждого региона из расчёта того, что выбираем мы лучшие предсказанные, а значения берём настоящие.

In [ ]:
profit(geo0_valid, geo0_pred, 200)

In [ ]:
profit(geo1_valid, geo1_pred, 200)

In [ ]:
profit(geo2_valid, geo2_pred, 200)

Выглядит аппетитно. Посмотрим так ли оно на самом деле и оценим риски уйти в минус.

## Расчёт прибыли и рисков 

In [ ]:
#создадим функцию для использвния технологии bootstrap при условии что мы можем рассмотреть только 500 скважим и выбрать из них 200 лучших
def bootstrap(target, probabilities):
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=SEED)
        probs_subsample = probabilities[target_subsample.index]
        values.append(profit(target_subsample, probs_subsample, 200))

    values = pd.Series(values)
    lower = values.quantile(0.025)
    higher = values.quantile(0.975)
    mean = values.mean()
    risk = st.percentileofscore(values, 0)
    
    print("Средняя выручка: {:2f} тыс. рублей".format(mean))
    print("2.5%-квантиль: {:2f} тыс. рублей".format(lower))
    print("97.5%-квантиль: {:2f} тыс. рублей".format(higher))
    print("Риск убытка в регионе", risk)

Выведем значения для каждого региона.

In [ ]:
bootstrap(geo0_valid, geo0_pred)

In [ ]:
bootstrap(geo1_valid, geo1_pred)

In [ ]:
bootstrap(geo2_valid, geo2_pred)

# Вывод

Как мы видим, в первом и третьем регионах риски выше 5% стать убыточными, что не подходит нам по условию задачи (не более 2.5%). Во втором же регионе риск составляет 0.2% и 97.5% того, что прибыль будет ~ 163 млн рублей. Модель смогла нам максимально точно в этом регионе предсказать какие скважины дадут нам больше сырья. Всё это связано с корреляцией почти в 1 с фичей f2, которая так точно подсказала модели "где нужно копать".